In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline

from astrobee_strips import STRIPS, Node, Queue, get_plan

from sys import path
path.append('/home/acauligi/Software')

import pdb
import numpy as np
from casadi import *

# STRIPS planner

In [2]:
strip = STRIPS()
n0 = Node(strip)
qq = Queue()
qq.queue(n0)

operators = ['dock_objA_dockA', 'dock_objA_dockB', 'dock_objA_dockC', \
            'dock_objB_dockA', 'dock_objB_dockB', 'dock_objB_dockC', \
            'undock_objA_dockA', 'undock_objA_dockB', 'undock_objA_dockC', \
            'undock_objB_dockA', 'undock_objB_dockB', 'undock_objB_dockC', \
            'grasp_objA', 'grasp_objB']

max_depth = 5

In [3]:
while not qq.is_empty():
    if all([node.depth >= max_depth for node in qq.list]):
        print('Max depth violated')
        break

    leaf_node = qq.dequeue()
    for operator in operators:
        new_leaf = leaf_node.act(operator)
        if all(new_leaf.strip.state == leaf_node.strip.state):
            continue
        qq.queue(new_leaf)

    if any([node.solved for node in qq.list]):
        print('solved')
        break

solved


In [4]:
solved_node = [node for node in qq.list if node.solved][0]
plan = get_plan(solved_node)
plan

['undock_objA_dockB', 'dock_objA_dockA', 'grasp_objB', 'dock_objB_dockB']

# TAMP Class


In [5]:
from astrobee_tamp import AstrobeeTAMP

mode = 'double_integrator'

if mode == 'nonlinear':
    Xi = np.array([3.,3.,3., 0.,0.,0.,1., 0.05,-0.05,0.02, 0.01,-0.01,0.01])
    Xref = DM([0.,0.1,0., 0.5,0.5,0.5,0.5, 0.,0.,0., 0.,0.,0.])
elif mode == 'double_integrator':
    Xi = np.array([3.,3., 0., 0.])
    Xref = DM([0.,0.1, 0., 0.])

at = AstrobeeTAMP(Xi, Xref, mode=mode)

at.plan = plan

In [6]:
at.unroll_plan()

at.construct_problem()
at.solve_problem()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     1076
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      264

Total number of variables............................:      236
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      158
                     variables with only upper bounds:        0
Total number of equa

True

In [7]:
at.soln['f']
at.soln['x'].shape
len(plan)
len(at.X)
len(at.U)

40